# Perform fine-tuning

In [1]:
!pip install datasets

     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------------------------------------- 341.8/341.8 kB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/521.2 kB ? eta -:--:--
   --------------------------------------- 521.2/521.2 kB 31.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/115.3 kB ? eta -:--:--
   ---------------------------------------- 115.3/115.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   --------------------------------------- 166.4/166.4 kB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/364.6 kB ? eta -:--:--
   --------------------------------------- 364.6/364.6 kB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.6 MB ? eta -:--:--
   -- ------------------------------------- 1.5/24.6 MB 48.6 MB/s eta 0:00:01
   ----- ---------------------------------- 3.1/24.6 MB 40.0 MB/s eta 0:00:01
   ------- -------------

In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# This dataset contains 2 sentence and will be labeled 0 and 1 based on similarity
def tokenize_func(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

C:\Users\fict\Documents\limjq_llm\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [6]:
raw_datasets["train"][0]['label']

1

In [7]:
print(raw_datasets["train"][0]['sentence1'])
print(raw_datasets["train"][0]['sentence2'])

Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .


In [8]:
print(raw_datasets["train"][1]['label'])
print(raw_datasets["train"][1]['sentence1'])
print(raw_datasets["train"][1]['sentence2'])

0
Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .


So, `1` means same meaning while `0` means different meaning. 

In [2]:
# map the data to suitable format for fine-tuning
# define the collator as well
tokenized_dataset = raw_datasets.map(tokenize_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|█████████████████████████████████████████████████████████████████| 408/408 [00:00<00:00, 14557.24 examples/s]


In [3]:
# now lets define the model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
!pip install accelerate -U

   ---------------------------------------- 0.0/265.7 kB ? eta -:--:--
   -------------------------------------- - 256.0/265.7 kB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 265.7/265.7 kB 4.1 MB/s eta 0:00:00


In [4]:
from transformers import TrainingArguments

train_args = TrainingArguments("train_classification1")

In [5]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [7]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [8]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.506900
1000,0.279000


TrainOutput(global_step=1377, training_loss=0.32450293802054964, metrics={'train_runtime': 94.3653, 'train_samples_per_second': 116.611, 'train_steps_per_second': 14.592, 'total_flos': 405626802939840.0, 'train_loss': 0.32450293802054964, 'epoch': 3.0})

In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [10]:
tokenized_dataset["train"][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0,
 'input_ids': [101,
  2572,
  3217,
  5831,
  5496,
  2010,
  2567,
  1010,
  3183,
  2002,
  2170,
  1000,
  1996,
  7409,
  1000,
  1010,
  1997,
  9969,
  4487,
  23809,
  3436,
  2010,
  3350,
  1012,
  102,
  7727,
  2000,
  2032,
  2004,
  2069,
  1000,
  1996,
  7409,
  1000,
  1010,
  2572,
  3217,
  5831,
  5496,
  2010,
  2567,
  1997,
  9969,
  4487,
  23809,
  3436,
  2010,
  3350,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
 

In [11]:
tokenized_dataset["validation"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 408
})

In [12]:
!pip install evaluate

   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [13]:
!pip install numpy

In [14]:
# define compute metrics
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
# redefine the training arguments, how metrics is computed and where is the directory
train_args = TrainingArguments("train_classification1", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
!pip install scikit-learn scipy

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.3 MB 15.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.0/9.3 MB 25.1 MB/s eta 0:00:01
   -------------------- ------------------- 4.7/9.3 MB 37.7 MB/s eta 0:00:01
   ------------------------------------ --- 8.6/9.3 MB 45.7 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 42.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
   -- ------------------------------------- 2.9/44.1 MB 63.0 MB/s eta 0:00:01
   ---- ----------------------------------- 5.2/44.1 MB 55.7 MB/s eta 0:00:01
   ------ --------------------------------- 7.2/44.1 MB 51.5 MB/s eta 0:00:01
   -------- ------------------------------- 9.1/44.1 MB 48.7 MB/s eta 0:00:01
   ---------

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.406799,0.862745,0.901060
2,0.333900,0.712412,0.828431,0.878472
3,0.170200,0.892179,0.838235,0.888514


TrainOutput(global_step=1377, training_loss=0.2069919228813487, metrics={'train_runtime': 102.5504, 'train_samples_per_second': 107.303, 'train_steps_per_second': 13.428, 'total_flos': 541038552024960.0, 'train_loss': 0.2069919228813487, 'epoch': 3.0})

In [23]:
raw_datasets["train"][1]['idx']

1

In [24]:
type(raw_datasets)

datasets.dataset_dict.DatasetDict

In [27]:
# new prediction (custom data)
new_data = tokenizer("I like apple.", "I love apple.", truncation=True, return_tensors="pt")
new_data

{'input_ids': tensor([[ 101, 1045, 2066, 6207, 1012,  102, 1045, 2293, 6207, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [29]:
model(new_data)

AttributeError: 

# full training

In [30]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

# load the dataset and define the tokenizer
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_func(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

# tokenized data and data collator
tokenized_dataset = raw_datasets.map(tokenize_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|███████████████████████████████████████████████████████████████| 1725/1725 [00:00<00:00, 20395.88 examples/s]


## Post processing of the tokenize data 

In [31]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset.column_names

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'validation': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}

In [32]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, collate_fn=data_collator, batch_size=8
)

eval_dataloader = DataLoader(
    tokenized_dataset["validation"], collate_fn=data_collator, batch_size=8 
)

In [33]:
for batch in train_dataloader:
    break

{k:v.shape for k,v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 70]),
 'token_type_ids': torch.Size([8, 70]),
 'attention_mask': torch.Size([8, 70])}

In [36]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6582, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [38]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

C:\Users\fict\Documents\limjq_llm\myenv\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
# check for cuda (if there is GPU acceleration)
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [40]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████▉| 1376/1377 [01:23<00:00, 16.85it/s]

In [41]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8651960784313726, 'f1': 0.9043478260869565}

100%|██████████████████████████████████████████████████████████████████████████████| 1377/1377 [01:41<00:00, 16.85it/s]

# New task (token classification)

## ner

In [42]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")






Generating train split: 100%|███████████████████████████████████████████| 14041/14041 [00:01<00:00, 9285.60 examples/s]

Generating validation split: 100%|████████████████████████████████████████| 3250/3250 [00:00<00:00, 8341.12 examples/s]

Generating test split: 100%|██████████████████████████████████████████████| 3453/3453 [00:00<00:00, 9319.05 examples/s]


In [43]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [44]:
idx_sample = 0
print(raw_datasets["train"][idx_sample]["id"])
print(raw_datasets["train"][idx_sample]["tokens"])
print(raw_datasets["train"][idx_sample]["pos_tags"])
print(raw_datasets["train"][idx_sample]["chunk_tags"])
print(raw_datasets["train"][idx_sample]["ner_tags"])

0
['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[22, 42, 16, 21, 35, 37, 16, 21, 7]
[11, 21, 11, 12, 21, 22, 11, 12, 0]
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [45]:
ner_features = raw_datasets["train"].features["ner_tags"]
ner_features

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [46]:
chunk_features = raw_datasets["train"].features["chunk_tags"]

In [47]:
chunk_features

Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)

In [48]:
label_names = ner_features.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [50]:
words = raw_datasets["train"][idx_sample]["tokens"]
labels = raw_datasets["train"][idx_sample]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


**Warning⚠️**
---
We have to pass a specific argument to `tokenizer` API when we are dealing with pre-tokenized text.

In [53]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████| 29.0/29.0 [00:00<?, ?B/s]

config.json: 100%|████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<?, ?B/s]

vocab.txt:   0%|                                                                            | 0.00/213k [00:00<?, ?B/s]
vocab.txt: 100%|█████████████████████████████████████████████████████████████████████| 213k/213k [00:00<00:00, 422kB/s]

tokenizer.json:   0%|                                                                       | 0.00/436k [00:00<?, ?B/s]
tokenizer.json: 100%|████████████████████████████████████████████████████████████████| 436k/436k [00:00<00:00, 590kB/s]


In [54]:
tokenizer.is_fast

True

In [55]:
inputs = tokenizer(raw_datasets["train"][idx_sample]["tokens"], is_split_into_words=True)
inputs

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [56]:
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [57]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]